<a href="https://colab.research.google.com/github/lopeslyra10/skillpath-ia/blob/main/skillpath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok

In [ ]:
!pip install -q google-genai

In [ ]:
!pip install fastapi uvicorn nest_asyncio google-genai

In [ ]:
import os
from google.colab import userdata
from google import genai
from fastapi import FastAPI
import nest_asyncio
import uvicorn
import threading
from uvicorn.config import Config
from uvicorn.server import Server
import requests
from pyngrok import ngrok

os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")

client = genai.Client()

In [ ]:
# COLOQUE SEU TOKEN AQUI
ngrok.set_auth_token("2xTF41FZfsOUMMiZysFY0rt7Vck_4Mbo7zJ5WJhX2G98woYZQ")

public_url = ngrok.connect(8000, "http")

print("API está rodando em:")
print(public_url)

API está rodando em:
NgrokTunnel: "https://226e93146301.ngrok-free.app" -> "http://localhost:8000"


# JSON gerado pelo Oracle

In [ ]:
import json

with open("dataset_skillpath.json", "r", encoding="utf-8") as f:
    dataset_json = json.load(f)

dataset_json[:2]

[{'id_usuario_final': 1,
  'nome_usuario': 'Augusto Lopes',
  'email': 'augustolopes@email.com',
  'id_trilha': 1,
  'nome_trilha': 'Front-End React Developer',
  'area': 'DESENVOLVIMENTO',
  'nivel': 'INICIANTE',
  'valor_nota': 5,
  'observacao': 'Trilha muito clara para iniciantes.'},
 {'id_usuario_final': 2,
  'nome_usuario': 'Felipe Ulson',
  'email': 'felipeulson@email.com',
  'id_trilha': 1,
  'nome_trilha': 'Front-End React Developer',
  'area': 'DESENVOLVIMENTO',
  'nivel': 'INICIANTE',
  'valor_nota': 4,
  'observacao': 'Conteúdos bons, faltou mais prática.'}]

In [ ]:
def enviar_para_java(prompt, resposta):
    url = "http://localhost:8080/ia/historico/external"

    payload = {
        "prompt": prompt,
        "resposta": resposta
    }

    try:
        requests.post(url, json=payload)
    except Exception as e:
        print("Erro ao enviar histórico para Java:", e)

def skillpath_ai(message):
    """
    Envia o texto do usuário + dataset Oracle
    para o modelo Gemini responder contextualizado.
    """

    prompt = f"""
    Você é a IA oficial da plataforma SkillPath.

    Você tem acesso ao dataset abaixo:
    {dataset_json}

    Responda sempre com clareza, empatia e em português.

    Se o usuário pedir:
    - Recomendação de trilhas → use area, nivel e notas
    - Recomendação baseada nas habilidades → considere padrões no dataset
    - Avaliação de compatibilidade → use id_usuario_final / id_trilha
    - Melhor trilha → escolha pela media de notas e alinhamento

    Agora, responda à pergunta do usuário:
    "{message}"
    """

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt
    )

    return response.text

In [ ]:
nest_asyncio.apply()

api = FastAPI(
    title="SkillPath IA API",
    description="API oficial da IA SkillPath integrando Gemini + Dataset Oracle",
    version="1.0.0"
)

@api.get("/")
def root():
    return {"status": "SkillPath IA funcionando!"}

@api.post("/ask")
def ask_ai(payload: dict):
    message = payload.get("prompt") or payload.get("message") or ""
    resposta = skillpath_ai(message)

    return {
        "user_message": message,
        "ia_response": resposta
    }


In [ ]:
# Função para executar o servidor Uvicorn
def run_uvicorn_thread(app):
    config = Config(app, host="0.0.0.0", port=8000, loop="asyncio")
    server = Server(config=config)
    server.run()

# Iniciar o Uvicorn em uma thread separada
uvicorn_thread = threading.Thread(target=run_uvicorn_thread, args=(api,))
uvicorn_thread.start()

print("FastAPI app está rodando em segundo plano em http://0.0.0.0:8000")
print("Você pode acessá-lo de fora do Colab usando serviços de tunelamento como o ngrok.")

FastAPI app está rodando em segundo plano em http://0.0.0.0:8000
Você pode acessá-lo de fora do Colab usando serviços de tunelamento como o ngrok.


INFO:     Started server process [1053]


In [ ]:
print("SkillPath IA iniciada! Digite 'sair' para encerrar.\n")

while True:
    user_input = input("Você: ")

    if user_input.lower() in ["sair", "exit", "quit"]:
        print("Encerrando o chat…")
        print("Chat encerrado. O skillpath agradece!")
        break

    resposta = skillpath_ai(user_input)
    print("\nSkillPath IA:", resposta, "\n")

SkillPath IA iniciada! Digite 'sair' para encerrar.

Você: Recomende trilhas de front

SkillPath IA: Olá! Com certeza posso te ajudar a encontrar trilhas na área de Front-End.

Analisando as trilhas disponíveis em nosso catálogo que se encaixam na sua busca, temos uma excelente opção para você:

*   **Trilha: Front-End React Developer**
    *   **Área:** DESENVOLVIMENTO
    *   **Nível:** INICIANTE

Esta trilha tem sido muito bem avaliada pelos nossos usuários, com uma média de **4.5 estrelas**. Ela é ideal para quem deseja iniciar ou aprofundar seus conhecimentos no desenvolvimento Front-End utilizando React.

Por exemplo, um de nossos usuários elogiou a trilha por ser "muito clara para iniciantes", destacando a facilidade de aprendizado. Outro usuário apreciou os "conteúdos bons", embora tenha sugerido mais prática, o que mostra um bom equilíbrio entre teoria e aplicabilidade.

É uma ótima escolha para quem busca uma base sólida e bem estruturada em Front-End. Se você tiver alguma pr